In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
import tensorflow_probability as tfp
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-06 23:08:39.911798: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 23:08:39.986831: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 23:08:39.989077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 23:08:39.989086: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

In [3]:
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

#### The data

In [4]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [5]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [6]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [7]:
def bnn(X_train, y_train, epochs=1000, optimizer='RMSProp'):
#     model = tf.keras.Sequential([
#     tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn, 
#                                 make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     ])
    
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
#     model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
#     model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(1, dtype=tf.float64), scale=1.0), 
                            reinterpreted_batch_ndims=1)
    model = tfk.Sequential([
        tfk.layers.InputLayer(input_shape=(3,), name='input'),
        tfk.layers.Dense(10, activation='relu', name='dense_1'),
        tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(1), activation=None, name='distribution_weights'),
        tfp.layers.MultivariateNormalTriL(1, activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/32), 
                                          name='output')], name='model')
    
#     model.compile(optimizer=optimizer, loss=neg_log_likelihood)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    return model

In [8]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     data_train =tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
#     data_train = data_train.batch(32).repeat(n_epochs)
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     data_test =tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
#     data_test = data_test.batch(1)
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    
    model = bnn(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [ ]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

2023-07-06 23:08:42.555571: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-06 23:08:42.555604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-06 23:08:42.555608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-06 23:08:42.555772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-06 23:08:42.555791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.116.04  Release Build  (dvs-builder@U16-I3-A16-3-3)  Thu Apr 27 18:11:06 UTC 2023
GCC version:  gcc 

Epoch 1/1000
289/289 [==============================] - 2s 2ms/step - loss: 3109176031.8090 - val_loss: 688011.0667
Epoch 2/1000
289/289 [==============================] - 0s 1ms/step - loss: 702001.8634 - val_loss: 691939.1310
Epoch 3/1000
289/289 [==============================] - 0s 1ms/step - loss: 695950.3233 - val_loss: 696005.1064
Epoch 4/1000
289/289 [==============================] - 0s 1ms/step - loss: 688318.0353 - val_loss: 691397.3683
Epoch 5/1000
289/289 [==============================] - 0s 1ms/step - loss: 681111.5587 - val_loss: 655830.1034
Epoch 6/1000
289/289 [==============================] - 0s 1ms/step - loss: 670567.1526 - val_loss: 691746.4384
Epoch 7/1000
289/289 [==============================] - 0s 1ms/step - loss: 656025.7310 - val_loss: 678006.3206
Epoch 8/1000
289/289 [==============================] - 0s 1ms/step - loss: 652087.0228 - val_loss: 656596.2931
Epoch 9/1000
289/289 [==============================] - 0s 1ms/step - loss: 649913.8403 - val_loss: 

289/289 [==============================] - 0s 1ms/step - loss: 499102.2814 - val_loss: 464990.6971
Epoch 74/1000
289/289 [==============================] - 0s 1ms/step - loss: 495782.0221 - val_loss: 506810.0148
Epoch 75/1000
289/289 [==============================] - 0s 1ms/step - loss: 493448.4516 - val_loss: 477802.1928
Epoch 76/1000
289/289 [==============================] - 0s 1ms/step - loss: 490422.0665 - val_loss: 524171.0948
Epoch 77/1000
289/289 [==============================] - 0s 1ms/step - loss: 488232.0868 - val_loss: 449940.2653
Epoch 78/1000
289/289 [==============================] - 0s 1ms/step - loss: 485057.4219 - val_loss: 446031.7316
Epoch 79/1000
289/289 [==============================] - 0s 1ms/step - loss: 482456.9182 - val_loss: 473174.0572
Epoch 80/1000
289/289 [==============================] - 0s 1ms/step - loss: 478966.0462 - val_loss: 517516.0882
Epoch 81/1000
289/289 [==============================] - 0s 1ms/step - loss: 475822.6030 - val_loss: 464154.82

289/289 [==============================] - 0s 1ms/step - loss: 352273.3588 - val_loss: 354192.0791
Epoch 146/1000
289/289 [==============================] - 0s 1ms/step - loss: 350561.1576 - val_loss: 335544.5109
Epoch 147/1000
289/289 [==============================] - 0s 1ms/step - loss: 349462.2067 - val_loss: 346809.2679
Epoch 148/1000
289/289 [==============================] - 0s 1ms/step - loss: 348979.1484 - val_loss: 358285.4871
Epoch 149/1000
289/289 [==============================] - 0s 1ms/step - loss: 347872.4510 - val_loss: 350885.9621
Epoch 150/1000
289/289 [==============================] - 0s 1ms/step - loss: 347008.4515 - val_loss: 341167.2151
Epoch 151/1000
289/289 [==============================] - 0s 1ms/step - loss: 345675.7912 - val_loss: 366133.4376
Epoch 152/1000
289/289 [==============================] - 0s 1ms/step - loss: 344969.1240 - val_loss: 322556.2044
Epoch 153/1000
289/289 [==============================] - 0s 1ms/step - loss: 343709.8877 - val_loss: 3

Epoch 217/1000
289/289 [==============================] - 1s 4ms/step - loss: 281085.6689 - val_loss: 276127.0951
Epoch 218/1000
289/289 [==============================] - 1s 4ms/step - loss: 280310.4481 - val_loss: 282348.6120
Epoch 219/1000
289/289 [==============================] - 1s 4ms/step - loss: 278602.6988 - val_loss: 293862.2031
Epoch 220/1000
289/289 [==============================] - 1s 4ms/step - loss: 278127.9976 - val_loss: 292997.9463
Epoch 221/1000
289/289 [==============================] - 1s 4ms/step - loss: 277448.4126 - val_loss: 275750.6436
Epoch 222/1000
289/289 [==============================] - 1s 4ms/step - loss: 276245.7846 - val_loss: 265033.9760
Epoch 223/1000
289/289 [==============================] - 1s 4ms/step - loss: 275081.0493 - val_loss: 252683.0665
Epoch 224/1000
289/289 [==============================] - 1s 4ms/step - loss: 273840.4360 - val_loss: 297496.0806
Epoch 225/1000
289/289 [==============================] - 1s 4ms/step - loss: 273559.229

Epoch 289/1000
289/289 [==============================] - 0s 1ms/step - loss: 209670.2315 - val_loss: 190506.6826
Epoch 290/1000
289/289 [==============================] - 0s 1ms/step - loss: 208883.5692 - val_loss: 222597.5979
Epoch 291/1000
289/289 [==============================] - 0s 1ms/step - loss: 208279.2350 - val_loss: 218681.1905
Epoch 292/1000
289/289 [==============================] - 0s 1ms/step - loss: 205917.5598 - val_loss: 265192.8284
Epoch 293/1000
289/289 [==============================] - 0s 1ms/step - loss: 206029.3138 - val_loss: 202399.5908
Epoch 294/1000
289/289 [==============================] - 0s 1ms/step - loss: 204238.3035 - val_loss: 226268.2932
Epoch 295/1000
289/289 [==============================] - 0s 1ms/step - loss: 203575.4153 - val_loss: 199149.5421
Epoch 296/1000
289/289 [==============================] - 0s 1ms/step - loss: 202862.0762 - val_loss: 241884.3587
Epoch 297/1000
289/289 [==============================] - 0s 1ms/step - loss: 201682.575

Epoch 361/1000
289/289 [==============================] - 0s 1ms/step - loss: 135884.8651 - val_loss: 142223.4800
Epoch 362/1000
289/289 [==============================] - 0s 1ms/step - loss: 134870.7127 - val_loss: 146218.5706
Epoch 363/1000
289/289 [==============================] - 0s 1ms/step - loss: 133415.3835 - val_loss: 146073.1730
Epoch 364/1000
289/289 [==============================] - 0s 1ms/step - loss: 132376.0804 - val_loss: 147841.1413
Epoch 365/1000
289/289 [==============================] - 0s 1ms/step - loss: 131521.0647 - val_loss: 146364.9002
Epoch 366/1000
289/289 [==============================] - 0s 1ms/step - loss: 130297.3353 - val_loss: 134893.9093
Epoch 367/1000
289/289 [==============================] - 0s 1ms/step - loss: 129810.9956 - val_loss: 127850.4860
Epoch 368/1000
289/289 [==============================] - 0s 1ms/step - loss: 128132.8042 - val_loss: 149203.1034
Epoch 369/1000
289/289 [==============================] - 0s 1ms/step - loss: 127033.702

289/289 [==============================] - 1s 4ms/step - loss: 59334.9291 - val_loss: 62867.3818
Epoch 434/1000
289/289 [==============================] - 1s 4ms/step - loss: 58983.1410 - val_loss: 66699.3559
Epoch 435/1000
289/289 [==============================] - 1s 4ms/step - loss: 58199.5185 - val_loss: 44764.6011
Epoch 436/1000
247/289 [========================>.....] - ETA: 0s - loss: 57688.1685

In [ ]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

In [ ]:
rmse_list

In [ ]:
mean_mape = np.mean(mape_list)
mean_mape

In [ ]:
mape_list